In [3]:
from transformers import pipeline
import pandas as pd

In [4]:
# Load your cleaned DataFrame
df = pd.read_csv('data/cleaned_bank_reviews.csv')


In [11]:
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="distilbert/distilbert-base-uncased-finetuned-sst-2-english",
    revision="714eb0f"
)


Device set to use cpu


In [12]:
def analyze_sentiment(review):
    try:
        result = sentiment_pipeline(review)[0]
        label = result['label']
        score = result['score']
        # Define neutral threshold (adjust as needed)
        if 0.4 < score < 0.6:
            label = "NEUTRAL"
        return label, score
    except Exception as e:
        print(f"Error analyzing sentiment for review: {review} | Error: {e}")
        return None, None


In [13]:
df['sentiment_label'], df['sentiment_score'] = zip(*df['review'].apply(analyze_sentiment))

# Save to CSV
df.to_csv('data/bank_reviews_with_sentiment.csv', index=False)


In [14]:
# Aggregate Sentiment Scores
sentiment_agg = df.groupby(['bank', 'rating']).agg({
    'sentiment_score': 'mean'
}).reset_index()

In [15]:
# Sort the aggregated results by sentiment score
sentiment_agg = sentiment_agg.sort_values(by='sentiment_score', ascending=False)
